# 0. Install Dependencies

# import data from json

In [1]:
from RoomModel import *

In [2]:
import json
# Opening JSON file
f = open('SampleData.json')

# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
Room =Room.from_dict(data)
  
# Closing file
f.close()

In [3]:
print(Room.furniture.dimensions)

[[107, 198], [102, 60], [50, 50]]


# 1. Test Random Environment with OpenAI Gym

In [4]:
from Room import *

In [5]:
# RoomEnv(Furniture Dimension ,Doors (doors Dimensions , (Door1 pos, Door2 pos,etc )),Windows(windows Dimensions,(window1 pos, window2 pos,etc )) )

In [6]:
env = RoomEnv(Room.furniture.dimensions,Room.doors.doors_dimensions,Room.doors.doors_positions,Room.windows.windows_dimensions,Room.windows.windows_positions,Room.furniture.keywords, Room.room_dimensions)

In [7]:
# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
#     switch = False
    
#     while not done:
#         env.render("human")
#         if(switch):
#             action = 0
#             switch=False
#         else:
#             action = 2
#             switch=True
# #         print('CurrentPos:{}'.format(action))
        
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{} '.format(episode, score))
#     env.close()

In [8]:
from rl.agents import DQNAgent
from rl.policy import  BoltzmannQPolicy
from rl.memory import SequentialMemory

# 2. Create a Deep Learning Model with Keras

In [9]:
from rl.agents import DQNAgent
from rl.policy import  *
from rl.memory import SequentialMemory
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.optimizers import Adam

In [10]:
states = env.observation_space.shape
actions = env.action_space.n

In [11]:

print(states)
print(actions)
print(env.step(0))
print(env.step(0))

(1, 2)
4
(array([117, 241]), 871, False, {})
(array([118, 241]), 870, False, {})


In [12]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=states))  
    model.add(Dense(128, activation='relu', input_shape=states))

    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [13]:
# del model 

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()
pt_depth = model.layers[0].get_input_shape_at(node_index=0)
print(pt_depth)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2)                 0         
_________________________________________________________________
dense (Dense)                (None, 128)               384       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 17,412
Trainable params: 17,412
Non-trainable params: 0
_________________________________________________________________
(None, 1, 2)


# 3. Build Agent with Keras-RL

In [16]:
print(env.state)


[118 241]


In [17]:
print(env.furniture_dimensions)

[[107, 198], [102, 60], [50, 50]]


In [18]:
def build_agent(model, actions):
    policy =  EpsGreedyQPolicy(eps=0.5)
    memory = SequentialMemory(limit=10000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=300, target_model_update= 1e-3)
    return dqn

In [19]:
def fitall(steps, dqns):
  for i in range(steps):
    for dqn in dqns:
        dqn.fit(env, nb_steps=steps, visualize=True, verbose=1)

In [20]:

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=True, verbose=1)

# fitall(100000,[dqn2,dqn])
env.close(); 

Training for 10000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 45s 4ms/step - reward: 1376.9124
done, took 44.899 seconds


In [21]:
env.nextPlease()
dqn2 = build_agent(model, actions)
dqn2.compile(Adam(lr=1e-3), metrics=['mae'])
dqn2.fit(env, nb_steps=10000, visualize=True, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: 821.3132
done, took 43.943 seconds


In [22]:
env.nextPlease()
dqn3 = build_agent(model, actions)
dqn3.compile(Adam(lr=1e-3), metrics=['mae'])
dqn3.fit(env, nb_steps=10000, visualize=True, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: -0.1774
done, took 45.172 seconds


In [23]:
print(env.state2)

[array([129, 388]), array([ 57, 260]), array([400, 462])]


In [24]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H%M%S")
dqn.save_weights('lastSaved2', overwrite=True)

In [25]:

scores = dqn.test(env, nb_episodes=5, visualize=True)
env.close()
print(scores.history['episode_reward'])

Testing for 5 episodes ...


KeyboardInterrupt: 

In [ ]:
print(env.colided)
env.step(0)
print(env.colided)

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('sequential_1doorWindowRandomStartSuccessdqn_weights2.h5f')

In [ ]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
env.close();
print(scores.history['episode_reward'])

In [ ]:
env.render3()

# 4. Reloading Agent from Memory

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)